In [12]:
import os
import pandas as pd

file_candidates = [
    "/Users/Yuhui/Desktop/data vis/data/Sample - Superstore.xls",
    "Sample - Superstore.xls"
]

file_path = None
for p in file_candidates:
    if os.path.exists(p):
        file_path = p
        break

if file_path is None:
    raise FileNotFoundError("Could not find 'Sample - Superstore.xls'. Please verify the file path and spelling.")

print("File found at:", file_path)

df = pd.read_excel(file_path)
print("Total rows:", len(df))
print("Columns:", list(df.columns))
df.head()

File found at: Sample - Superstore.xls
Total rows: 9994
Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-11-08,2016-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-06-12,2016-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-10-11,2015-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [13]:
sales = pd.to_numeric(df["Sales"], errors="coerce").dropna()

mean_val = sales.mean()
median_val = sales.median()
mode_series = sales.mode()
mode_val = mode_series.iloc[0] if not mode_series.empty else None

print("Count:", len(sales))
print("Mean:", round(mean_val, 2))
print("Median:", round(median_val, 2))
print("Mode:", round(mode_val, 2) if mode_val is not None else None)

Count: 9994
Mean: 229.86
Median: 54.49
Mode: 12.96


In [14]:
# Export the Sales column as a CSV file
hardway_csv = "superstore_sales_only.csv"
pd.DataFrame({"Sales": sales}).to_csv(hardway_csv, index=False)
print("CSV file saved:", hardway_csv)

CSV file saved: superstore_sales_only.csv


In [15]:
import csv

csv_path = "superstore_sales_only.csv"

# Read values from CSV using only the standard library
values = []
with open(csv_path, "r", newline="", encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        raw = row.get("Sales", "").strip()
        try:
            x = float(raw)
            values.append(x)
        except:
            pass

# Define mean, median, mode manually
def my_mean(nums):
    n = 0
    total = 0.0
    for x in nums:
        n += 1
        total += x
    return total / n if n else None

def my_median(nums):
    n = len(nums)
    if n == 0:
        return None
    arr = sorted(nums)
    mid = n // 2
    if n % 2 == 1:
        return arr[mid]
    else:
        return (arr[mid - 1] + arr[mid]) / 2

def my_mode(nums):
    if not nums:
        return None
    counts = {}
    for x in nums:
        counts[x] = counts.get(x, 0) + 1
    max_cnt = 0
    mode_val = None
    for k, v in counts.items():
        if v > max_cnt:
            max_cnt = v
            mode_val = k
    return mode_val

print("Count:", len(values))
print("Mean:", round(my_mean(values), 2))
print("Median:", round(my_median(values), 2))
m = my_mode(values)
print("Mode:", round(m, 2) if m is not None else None)

Count: 9994
Mean: 229.86
Median: 54.49
Mode: 12.96


In [16]:
# Prepare data using pandas (allowed for data prep)
import pandas as pd

bin_edges = [0, 25, 50, 100, 200, 500, 1000, 2000, 5000, float("inf")]
bin_labels = ["0-25", "25-50", "50-100", "100-200", "200-500", "500-1k", "1k-2k", "2k-5k", "5k+"]

binned = pd.cut(sales, bins=bin_edges, labels=bin_labels, right=False)
counts = binned.value_counts().reindex(bin_labels, fill_value=0)

bucket_counts = [(label, int(counts[label])) for label in bin_labels]

# Draw ASCII chart using standard library only
max_count = max(c for _, c in bucket_counts)
max_bar_width = 60

def make_bar(count, max_count, max_width):
    if max_count == 0:
        return ""
    length = int(round(count / max_count * max_width))
    return "#" * length

print("Sales Distribution (USD ranges)")
for label, cnt in bucket_counts:
    bar = make_bar(cnt, max_count, max_bar_width)
    print(f"{label:>7} | {bar} ({cnt})")


Sales Distribution (USD ranges)
   0-25 | ############################################################ (3308)
  25-50 | ############################ (1541)
 50-100 | ######################### (1377)
100-200 | ###################### (1189)
200-500 | ########################## (1417)
 500-1k | ############# (694)
  1k-2k | ###### (328)
  2k-5k | ## (121)
    5k+ |  (19)


# Project 1 – Exploring Sales in the Superstore Dataset

**Author:** Yuhui Huang  
**Dataset:** Sample – Superstore.xls (Tableau sample data)  
**Numeric column analyzed:** `Sales` (USD)

---

## General Project Information

### Overview
This project uses the public **Sample – Superstore** dataset to practice basic data analysis tasks in Python.  
The goal is to compute the **mean**, **median**, and **mode** of the variable `Sales` using two different approaches:
1. The *pandas* library (the easy way)  
2. The Python standard library only (the hard way)  

A simple ASCII-style visualization is then created to show the distribution of `Sales` values.

### Why this dataset
- Public, non-sensitive, and widely used for analytics exercises  
- Contains multiple numeric columns (`Sales`, `Profit`, `Quantity`, `Discount`)  
- `Sales` is continuous and well suited for summary-statistics practice

### Methods and Reproducibility
All computations are dynamic and will continue to work if the dataset size or values change.  
Missing or invalid values in `Sales` are safely removed before analysis.  
The notebook contains three major steps:
1. Compute mean, median, and mode with *pandas*  
2. Recompute the same statistics manually with pure Python code  
3. Create an ASCII visualization using only standard-library functions  

### Assumptions and Caveats
- Rows with non-numeric `Sales` values are excluded.  
- This is a practice exercise; results are illustrative rather than scientifically rigorous.  
- Visualization is text-based for demonstration purposes.

### Key Takeaways
- Writing manual calculations deepens understanding of statistical concepts.  
- ASCII visualizations can make numeric patterns quick to grasp without external libraries.  
- The workflow illustrates how to analyze real-world data programmatically and cleanly.